In [2]:
import requests
import datetime
import time
import csv
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint
from imblearn.over_sampling import SMOTE
pd.set_option('display.max_columns', None)
plt.style.use('seaborn')
import xgboost as xgb
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier
import seaborn as sns

Using TensorFlow backend.


In [3]:
newmodels_df = pd.read_csv('newmodels_df.csv')
newmodels_df.drop('Unnamed: 0', axis=1, inplace=True)
newmodels_df

,fight,winner,loser,date,cumultime_w,cumulStr_Att_fighter_w,cumulStr_Att_oponent_w,cumulTD_Att_fighter_w,cumulTD_Att_oponent_w,KDPM_w,KDPM_ratio_w,SLPM_w,SLPM_ratio_w,SAPM_w,SAPM_ratio_w,Str_Acc_w,Str_Def_w,TD_Acc_w,TD_Def_w,SubPM_w,PassPM_w,RevPM_w,Head_SLPM_w,Body_SLPM_w,Leg_SLPM_w,Dist_SLPM_w,Clinch_SLPM_w,Ground_SLPM_w,wins_w,losses_w,cumultime_l,cumulStr_Att_fighter_l,cumulStr_Att_oponent_l,cumulTD_Att_fighter_l,cumulTD_Att_oponent_l,KDPM_l,KDPM_ratio_l,SLPM_l,SLPM_ratio_l,SAPM_l,SAPM_ratio_l,Str_Acc_l,Str_Def_l,TD_Acc_l,TD_Def_l,SubPM_l,PassPM_l,RevPM_l,Head_SLPM_l,Body_SLPM_l,Leg_SLPM_l,Dist_SLPM_l,Clinch_SLPM_l,Ground_SLPM_l,wins_l,losses_l,implied_prob_w,implied_prob_l,prev_w,prev_l,bday_w,height_w,weight_w,reach_w,stance_w,bday_l,height_l,weight_l,reach_l,stance_l,age_w,age_l,weightclass,location
0,Joanna Jedrzejczyk vs Michelle Waterson,Joanna-Jedrzejczyk,Michelle-Waterson,2019-10-12,89.700000,2662.0,2007.0,50.0,66.0,0.003655,0.000000,1.711585,0.272172,0.970899,0.241132,0.001916,0.003950,0.041355,0.097502,0.001593,0.004778,0.000000,0.830364,0.407892,0.473329,1.371245,0.290622,0.049718,9,3,39.766667,516.0,667.0,14.0,16.0,0.000000,0.000000,1.125275,0.143832,1.541284,0.364421,0.007252,0.004489,0.244048,0.171875,0.005427,0.021554,0.003592,0.495293,0.354215,0.275767,0.666664,0.290306,0.168304,5,2,0.746269,0.270270,12,7,1987-08-18,66.0,125.0,65.0,Orthodox,1986-01-06,63.0,115.0,62.0,Orthodox,32.0,33.0,Women's Strawweight,"Tampa, Florida, USA"
1,Michelle Waterson vs Rose Namajunas,Rose-Namajunas,Michelle-Waterson,2017-04-15,26.633333,603.0,428.0,18.0,16.0,0.000000,0.000000,1.512411,0.353095,1.382370,0.178646,0.003308,0.006544,0.146296,0.184524,0.036896,0.119628,0.000000,0.969301,0.407205,0.135905,0.959987,0.343305,0.209119,3,2,7.983333,36.0,77.0,2.0,4.0,0.000000,0.000000,0.403235,0.073513,1.719329,1.036534,0.020833,0.006634,0.500000,0.000000,0.027035,0.000000,0.000000,0.118496,0.128852,0.155887,0.311774,0.091461,0.000000,2,0,0.469484,0.523560,5,2,1992-06-29,65.0,115.0,65.0,Orthodox,1986-01-06,63.0,115.0,62.0,Orthodox,24.0,31.0,Women's Strawweight,"Kansas City, Missouri, USA"
2,Joanna Jedrzejczyk vs Tecia Torres,Joanna-Jedrzejczyk,Tecia-Torres,2018-07-28,73.700000,2246.0,1738.0,50.0,42.0,0.004449,0.000000,1.845392,0.293876,0.983534,0.270777,0.001965,0.003922,0.041355,0.114103,0.001938,0.005815,0.000000,0.951836,0.377773,0.515782,1.502454,0.282427,0.060512,8,2,50.883333,1065.0,973.0,31.0,28.0,0.002808,0.000000,1.591181,0.292761,0.972255,0.101005,0.003815,0.005568,0.102151,0.148810,0.010435,0.038511,0.000000,0.975857,0.322495,0.292829,1.240189,0.250716,0.100276,6,2,0.735294,0.266667,10,8,1987-08-18,66.0,125.0,65.0,Orthodox,1989-08-16,61.0,115.0,60.0,Orthodox,30.0,28.0,Women's Strawweight,"Calgary, Alberta, Canada"
3,Tecia Torres vs Weili Zhang,Weili-Zhang,Tecia-Torres,2019-03-02,10.683333,181.0,206.0,4.0,3.0,0.000000,0.000000,1.598222,0.509350,1.258356,0.230706,0.005814,0.004141,0.083333,0.500000,0.064198,0.050826,0.000000,1.209043,0.284865,0.104313,0.490769,0.116354,0.991098,2,0,57.883333,1143.0,1139.0,41.0,28.0,0.002468,0.000000,1.480200,0.266552,1.007694,0.121249,0.003925,0.005307,0.077236,0.148810,0.009173,0.033853,0.000000,0.884992,0.288431,0.306777,1.161782,0.230268,0.088150,6,3,0.555556,0.416667,2,9,1989-08-13,64.0,115.0,63.0,Switch,1989-08-16,61.0,115.0,60.0,Orthodox,29.0,29.0,Women's Strawweight,"Las Vegas, Nevada, USA"
4,Jessica Andrade vs Tecia Torres,Jessica-Andrade,Tecia-Torres,2018-02-24,69.933333,1375.0,1768.0,48.0,34.0,0.000000,0.000000,1.326570,0.196215,2.191892,0.322483,0.004438,0.003312,0.126562,0.110399,0.030354,0.046400,0.000000,0.894185,0.300973,0.131412,0.765028,0.370830,0.190712,8,4,43.883333,915.0,795.0,29.0,16.0,0.003255,0.000000,1.620375,0.322370,0.827847,0.086733,0.003937,0.006207,0.074713,0.250000,0.012100,0.041398,0.000000,0.955729,0.354405,0.310241,1.239437,0.274432,0.106506,6,1,0.694444,0.294118,12,7,1991-09-25,61.0,115.0,62.0,Orthodox,1989-08-16,61.0,115.0,60.0,Orthodox,26.0,28.0,Women's Straww

In [4]:
country = []
for i in list(newmodels_df['location']):
    country.append(i[-5:])
country

[', USA',
 ', USA',
 'anada',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 'China',
 ', USA',
 ', USA',
 'razil',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 'Japan',
 'anada',
 'ralia',
 'anada',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 'ralia',
 'razil',
 'razil',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 'oland',
 'anada',
 ', USA',
 'razil',
 ', USA',
 ', USA',
 'China',
 ', USA',
 ', USA',
 ', USA',
 'aland',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 'ntina',
 ', USA',
 'ngdom',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 'eland',
 'ralia',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 ', USA',
 'anada',
 'razil',
 ', USA',
 ', USA',
 'ralia',
 ', USA',
 ', USA',
 'exico',
 ', USA',
 'ralia',
 'razil',
 ', USA',
 ', USA',
 ', USA',
 'ngdom',
 ', USA',
 'ralia',
 ', USA',
 ', USA',
 'ralia',


In [5]:
newmodels_df['country'] = country

In [6]:
newest = pd.get_dummies(data=newmodels_df, columns=['stance_w', 'stance_l', 'weightclass'])

In [7]:
newest = newest.sample(frac=1)

In [8]:
newest

,fight,winner,loser,date,cumultime_w,cumulStr_Att_fighter_w,cumulStr_Att_oponent_w,cumulTD_Att_fighter_w,cumulTD_Att_oponent_w,KDPM_w,KDPM_ratio_w,SLPM_w,SLPM_ratio_w,SAPM_w,SAPM_ratio_w,Str_Acc_w,Str_Def_w,TD_Acc_w,TD_Def_w,SubPM_w,PassPM_w,RevPM_w,Head_SLPM_w,Body_SLPM_w,Leg_SLPM_w,Dist_SLPM_w,Clinch_SLPM_w,Ground_SLPM_w,wins_w,losses_w,cumultime_l,cumulStr_Att_fighter_l,cumulStr_Att_oponent_l,cumulTD_Att_fighter_l,cumulTD_Att_oponent_l,KDPM_l,KDPM_ratio_l,SLPM_l,SLPM_ratio_l,SAPM_l,SAPM_ratio_l,Str_Acc_l,Str_Def_l,TD_Acc_l,TD_Def_l,SubPM_l,PassPM_l,RevPM_l,Head_SLPM_l,Body_SLPM_l,Leg_SLPM_l,Dist_SLPM_l,Clinch_SLPM_l,Ground_SLPM_l,wins_l,losses_l,implied_prob_w,implied_prob_l,prev_w,prev_l,bday_w,height_w,weight_w,reach_w,bday_l,height_l,weight_l,reach_l,age_w,age_l,location,country,stance_w_Orthodox,stance_w_Southpaw,stance_w_Switch,stance_l_Orthodox,stance_l_Southpaw,stance_l_Switch,weightclass_Bantamweight,weightclass_Catch Weight,weightclass_Featherweight,weightclass_Flyweight,weightclass_Heavyweight,weightclass_Light Heavyweight,weightclass_Lightweight,weightclass_Middleweight,weightclass_Welterweight,weightclass_Women's Bantamweight,weightclass_Women's Featherweight,weightclass_Women's Flyweight,weightclass_Women's Strawweight
1512,Leandro Silva vs Charlie Brenneman,Leandro-Silva,Charlie-Brenneman,2014-11-08,14.000000,127.0,159.0,14.0,5.0,0.000000,0.000000,0.775510,0.159812,0.704082,0.146044,0.009643,0.006409,0.035714,0.200000,0.010204,0.040816,0.000000,0.346939,0.132653,0.295918,0.683673,0.091837,0.000000,0,2,40.883333,231.0,229.0,48.0,11.0,0.013977,0.000000,0.846688,0.422996,1.338457,0.651842,0.020701,0.019675,0.064608,0.500000,0.013977,0.148752,0.003494,0.405448,0.335118,0.106122,0.234423,0.149987,0.462278,4,6,0.609756,0.400000,2,10,1985-11-11,69.0,155.0,72.0,1981-02-09,70.0,155.0,72.0,28.0,33.0,"Uberlandia, Minas Gerais, Brazil",razil,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
294,Jon Jones vs Daniel Cormier,Jon-Jones,Daniel-Cormier,2015-01-03,82.200000,1419.0,1168.0,53.0,35.0,0.007096,0.000000,1.539772,0.453875,0.845349,0.194068,0.005770,0.008242,0.141967,0.291667,0.023926,0.051751,0.000000,0.864081,0.314560,0.361131,0.862241,0.246077,0.431453,14,1,21.200000,325.0,218.0,8.0,5.0,0.035825,0.000000,1.706107,0.723729,0.846836,0.153420,0.006189,0.011381,0.200000,0.600000,0.016035,0.088193,0.000000,1.224436,0.373854,0.107817,0.660546,0.327374,0.718187,4,0,0.609756,0.400000,15,4,1987-07-19,76.0,205.0,84.0,1979-03-20,71.0,235.0,72.0,27.0,35.0,"Las Vegas, Nevada, USA",", USA",1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
63,Marlon Vera vs Frankie Saenz,Marlon-Vera,Frankie-Saenz,2019-03-23,54.783333,807.0,964.0,25.0,26.0,0.005710,0.000000,1.211761,0.180450,1.440506,0.232817,0.005771,0.005252,0.123333,0.175916,0.021664,0.042691,0.000000,0.652580,0.309733,0.249448,0.780882,0.165348,0.265530,6,4,52.900000,879.0,908.0,26.0,76.0,0.004847,0.000000,1.137955,0.137926,1.282121,0.193901,0.004096,0.004561,0.042735,0.066829,0.005401,0.010802,0.002701,0.677344,0.317483,0.143127,0.799906,0.291101,0.046947,5,3,0.617284,0.400000,10,8,1992-12-02,68.0,135.0,70.0,1980-08-12,66.0,135.0,66.0,26.0,38.0,"Nashville, Tennessee, USA",", USA",0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1312,Alex Oliveira vs James Moontasri,Alex-Oliveira,James-Moontasri,2016-07-23,21.483333,253.0,173.0,20.0,16.0,0.016333,0.000000,1.345555,0.427501,0.989875,0.197760,0.009170,0.015141,0.015625,0.187500,0.013299,0.056231,0.000000,0.888153,0.246254,0.211148,0.772143,0.527448,0.045964,3,2,17.350000,169.0,122.0,0.0,23.0,0.020764,0.000000,1.198715,0.321112,0.816544,0.233373,0.011168,0.016631,NaN,0.124638,0.020463,0.028697,0.000000,0.849117,0.212950,0.136648,0.411018,0.286780,0.500917,2,2,0.666667,0.347222,5,4,1988-02-21,71.0,170.0,76.0,1988-04-10,70.0,170.0,71.0,28.0,28.0,"Chicago, Illinois, USA",", USA",1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
13,Karolina Kowalkiewicz vs Michelle Waterson,Michelle-Waterson,Karolina-Kowalkiewicz,2019-03-30,32.766667,398.0,518.0,14.0,12.0,0.000000,0.000000,1.11715

In [9]:
newest.columns

Index(['fight', 'winner', 'loser', 'date', 'cumultime_w',
       'cumulStr_Att_fighter_w', 'cumulStr_Att_oponent_w',
       'cumulTD_Att_fighter_w', 'cumulTD_Att_oponent_w', 'KDPM_w',
       'KDPM_ratio_w', 'SLPM_w', 'SLPM_ratio_w', 'SAPM_w', 'SAPM_ratio_w',
       'Str_Acc_w', 'Str_Def_w', 'TD_Acc_w', 'TD_Def_w', 'SubPM_w', 'PassPM_w',
       'RevPM_w', 'Head_SLPM_w', 'Body_SLPM_w', 'Leg_SLPM_w', 'Dist_SLPM_w',
       'Clinch_SLPM_w', 'Ground_SLPM_w', 'wins_w', 'losses_w', 'cumultime_l',
       'cumulStr_Att_fighter_l', 'cumulStr_Att_oponent_l',
       'cumulTD_Att_fighter_l', 'cumulTD_Att_oponent_l', 'KDPM_l',
       'KDPM_ratio_l', 'SLPM_l', 'SLPM_ratio_l', 'SAPM_l', 'SAPM_ratio_l',
       'Str_Acc_l', 'Str_Def_l', 'TD_Acc_l', 'TD_Def_l', 'SubPM_l', 'PassPM_l',
       'RevPM_l', 'Head_SLPM_l', 'Body_SLPM_l', 'Leg_SLPM_l', 'Dist_SLPM_l',
       'Clinch_SLPM_l', 'Ground_SLPM_l', 'wins_l', 'losses_l',
       'implied_prob_w', 'implied_prob_l', 'prev_w', 'prev_l', 'bday_w',
       'heig

In [38]:
winners = newest[['fight', 'date', 'winner', 'cumultime_w',
       'cumulStr_Att_fighter_w', 'cumulStr_Att_oponent_w',
       'cumulTD_Att_fighter_w', 'cumulTD_Att_oponent_w', 'KDPM_w',
       'KDPM_ratio_w', 'SLPM_w', 'SLPM_ratio_w', 'SAPM_w', 'SAPM_ratio_w',
       'Str_Acc_w', 'Str_Def_w', 'TD_Acc_w', 'TD_Def_w', 'SubPM_w', 'PassPM_w',
       'RevPM_w', 'Head_SLPM_w', 'Body_SLPM_w', 'Leg_SLPM_w', 'Dist_SLPM_w',
       'Clinch_SLPM_w', 'Ground_SLPM_w', 'wins_w', 'losses_w', 'implied_prob_w',
       'prev_w', 'height_w', 'weight_w', 'reach_w', 'age_w', 'stance_w_Orthodox',
       'stance_w_Southpaw', 'stance_w_Switch']]

In [39]:
losers = newest[['fight', 'date', 'loser', 'cumultime_l',
       'cumulStr_Att_fighter_l', 'cumulStr_Att_oponent_l',
       'cumulTD_Att_fighter_l', 'cumulTD_Att_oponent_l', 'KDPM_l',
       'KDPM_ratio_l', 'SLPM_l', 'SLPM_ratio_l', 'SAPM_l', 'SAPM_ratio_l',
       'Str_Acc_l', 'Str_Def_l', 'TD_Acc_l', 'TD_Def_l', 'SubPM_l', 'PassPM_l',
       'RevPM_l', 'Head_SLPM_l', 'Body_SLPM_l', 'Leg_SLPM_l', 'Dist_SLPM_l',
       'Clinch_SLPM_l', 'Ground_SLPM_l', 'wins_l', 'losses_l', 'implied_prob_l',
       'prev_l', 'height_l', 'weight_l', 'reach_l', 'age_l', 'stance_l_Orthodox',
       'stance_l_Southpaw', 'stance_l_Switch']]

In [40]:
len(winners)

1555

In [41]:
len(losers)

1555

In [42]:
winners.rename(columns={'winner': 'fighter'}, inplace=True)
losers.rename(columns={'loser': 'fighter'}, inplace=True)

In [43]:
winners

,fight,date,fighter,cumultime_w,cumulStr_Att_fighter_w,cumulStr_Att_oponent_w,cumulTD_Att_fighter_w,cumulTD_Att_oponent_w,KDPM_w,KDPM_ratio_w,SLPM_w,SLPM_ratio_w,SAPM_w,SAPM_ratio_w,Str_Acc_w,Str_Def_w,TD_Acc_w,TD_Def_w,SubPM_w,PassPM_w,RevPM_w,Head_SLPM_w,Body_SLPM_w,Leg_SLPM_w,Dist_SLPM_w,Clinch_SLPM_w,Ground_SLPM_w,wins_w,losses_w,implied_prob_w,prev_w,height_w,weight_w,reach_w,age_w,stance_w_Orthodox,stance_w_Southpaw,stance_w_Switch
1512,Leandro Silva vs Charlie Brenneman,2014-11-08,Leandro-Silva,14.000000,127.0,159.0,14.0,5.0,0.000000,0.000000,0.775510,0.159812,0.704082,0.146044,0.009643,0.006409,0.035714,0.200000,0.010204,0.040816,0.000000,0.346939,0.132653,0.295918,0.683673,0.091837,0.000000,0,2,0.609756,2,69.0,155.0,72.0,28.0,1,0,0
294,Jon Jones vs Daniel Cormier,2015-01-03,Jon-Jones,82.200000,1419.0,1168.0,53.0,35.0,0.007096,0.000000,1.539772,0.453875,0.845349,0.194068,0.005770,0.008242,0.141967,0.291667,0.023926,0.051751,0.000000,0.864081,0.314560,0.361131,0.862241,0.246077,0.431453,14,1,0.609756,15,76.0,205.0,84.0,27.0,1,0,0
63,Marlon Vera vs Frankie Saenz,2019-03-23,Marlon-Vera,54.783333,807.0,964.0,25.0,26.0,0.005710,0.000000,1.211761,0.180450,1.440506,0.232817,0.005771,0.005252,0.123333,0.175916,0.021664,0.042691,0.000000,0.652580,0.309733,0.249448,0.780882,0.165348,0.265530,6,4,0.617284,10,68.0,135.0,70.0,26.0,0,0,1
1312,Alex Oliveira vs James Moontasri,2016-07-23,Alex-Oliveira,21.483333,253.0,173.0,20.0,16.0,0.016333,0.000000,1.345555,0.427501,0.989875,0.197760,0.009170,0.015141,0.015625,0.187500,0.013299,0.056231,0.000000,0.888153,0.246254,0.211148,0.772143,0.527448,0.045964,3,2,0.666667,5,71.0,170.0,76.0,28.0,1,0,0
13,Karolina Kowalkiewicz vs Michelle Waterson,2019-03-30,Michelle-Waterson,32.766667,398.0,518.0,14.0,12.0,0.000000,0.000000,1.117159,0.147378,1.591522,0.408006,0.008188,0.004679,0.244048,0.187500,0.006587,0.026159,0.004360,0.531346,0.342690,0.243123,0.625972,0.295647,0.195539,4,2,0.408163,6,63.0,115.0,62.0,33.0,1,0,0
721,Joseph Benavidez vs Dustin Ortiz,2019-01-19,Joseph-Benavidez,92.716667,2030.0,1482.0,53.0,77.0,0.016273,0.000000,1.463462,0.333259,0.873614,0.124406,0.003085,0.006528,0.072216,0.143267,0.011724,0.069795,0.002609,0.835980,0.368797,0.258684,1.123083,0.186038,0.154342,13,3,0.787402,16,64.0,125.0,65.0,34.0,0,1,0
69,Urijah Faber vs Brad Pickett,2016-12-17,Urijah-Faber,91.066667,1641.0,1694.0,69.0,57.0,0.011275,0.000000,1.241575,0.366832,0.958663,0.153747,0.004216,0.005889,0.059213,0.111914,0.022423,0.027406,0.006887,0.890280,0.230164,0.121131,0.792866,0.161124,0.287584,9,6,0.757576,15,66.0,135.0,67.0,37.0,1,0,0
1214,Sergio Moraes vs Omari Akhmedov,2015-12-10,Sergio-Moraes,22.383333,220.0,327.0,17.0,0.0,0.000000,0.000000,0.886933,0.199362,0.890069,0.187206,0.009764,0.007996,0.119430,NaN,0.022536,0.078108,0.000000,0.693928,0.115293,0.077712,0.699962,0.046941,0.140030,3,1,0.444444,4,72.0,170.0,72.0,33.0,1,0,0
964,Elizeu Zaleski dos Santos vs Curtis Millender,2019-03-09,Elizeu-Zaleski-dos-Santos,39.050000,774.0,761.0,26.0,23.0,0.017959,0.000000,1.453954,0.276499,1.147351,0.159301,0.003697,0.005982,0.093590,0.192547,0.005071,0.035750,0.005071,0.945633,0.306452,0.201869,0.847619,0.464244,0.142091,6,1,0.476190,7,71.0,170.0,73.0,32.0,1,0,0
171,Nikita Krylov vs Misha Cirkunov,2016-12-10,Misha-Cirkunov,10.583333,186.0,180.0,13.0,1.0,0.000000,0.000000,2.938102,0.740157,1.405829,0.122405,0.009697,0.011383,0.121795,1.000000,0.066372,0.319057,0.000000,2.289252,0.505866,0.142984,1.791165,0.732157,0.414780,3,0,0.531915,3,75.0,205.0,77.0,29.0,1,0,0


In [44]:
losers

,fight,date,fighter,cumultime_l,cumulStr_Att_fighter_l,cumulStr_Att_oponent_l,cumulTD_Att_fighter_l,cumulTD_Att_oponent_l,KDPM_l,KDPM_ratio_l,SLPM_l,SLPM_ratio_l,SAPM_l,SAPM_ratio_l,Str_Acc_l,Str_Def_l,TD_Acc_l,TD_Def_l,SubPM_l,PassPM_l,RevPM_l,Head_SLPM_l,Body_SLPM_l,Leg_SLPM_l,Dist_SLPM_l,Clinch_SLPM_l,Ground_SLPM_l,wins_l,losses_l,implied_prob_l,prev_l,height_l,weight_l,reach_l,age_l,stance_l_Orthodox,stance_l_Southpaw,stance_l_Switch
1512,Leandro Silva vs Charlie Brenneman,2014-11-08,Charlie-Brenneman,40.883333,231.0,229.0,48.0,11.0,0.013977,0.000000,0.846688,0.422996,1.338457,0.651842,0.020701,0.019675,0.064608,0.500000,0.013977,0.148752,0.003494,0.405448,0.335118,0.106122,0.234423,0.149987,0.462278,4,6,0.400000,10,70.0,155.0,72.0,33.0,1,0,0
294,Jon Jones vs Daniel Cormier,2015-01-03,Daniel-Cormier,21.200000,325.0,218.0,8.0,5.0,0.035825,0.000000,1.706107,0.723729,0.846836,0.153420,0.006189,0.011381,0.200000,0.600000,0.016035,0.088193,0.000000,1.224436,0.373854,0.107817,0.660546,0.327374,0.718187,4,0,0.400000,4,71.0,235.0,72.0,35.0,1,0,0
63,Marlon Vera vs Frankie Saenz,2019-03-23,Frankie-Saenz,52.900000,879.0,908.0,26.0,76.0,0.004847,0.000000,1.137955,0.137926,1.282121,0.193901,0.004096,0.004561,0.042735,0.066829,0.005401,0.010802,0.002701,0.677344,0.317483,0.143127,0.799906,0.291101,0.046947,5,3,0.400000,8,66.0,135.0,66.0,38.0,1,0,0
1312,Alex Oliveira vs James Moontasri,2016-07-23,James-Moontasri,17.350000,169.0,122.0,0.0,23.0,0.020764,0.000000,1.198715,0.321112,0.816544,0.233373,0.011168,0.016631,NaN,0.124638,0.020463,0.028697,0.000000,0.849117,0.212950,0.136648,0.411018,0.286780,0.500917,2,2,0.347222,4,70.0,170.0,71.0,28.0,0,0,1
13,Karolina Kowalkiewicz vs Michelle Waterson,2019-03-30,Karolina-Kowalkiewicz,49.016667,1400.0,1415.0,22.0,24.0,0.000000,0.000000,1.809788,0.173745,1.916593,0.220659,0.002191,0.003358,0.042929,0.145833,0.000000,0.014572,0.002914,1.259514,0.347847,0.202426,1.322359,0.452455,0.034974,5,3,0.591716,8,63.0,115.0,64.0,33.0,1,0,0
721,Joseph Benavidez vs Dustin Ortiz,2019-01-19,Dustin-Ortiz,75.450000,998.0,881.0,97.0,56.0,0.059087,0.000000,1.359259,0.389158,0.847364,0.164900,0.005982,0.008766,0.036731,0.098335,0.020425,0.058558,0.013959,0.946778,0.257063,0.155418,0.609414,0.220994,0.528851,8,5,0.229885,13,65.0,125.0,65.0,30.0,1,0,0
69,Urijah Faber vs Brad Pickett,2016-12-17,Brad-Pickett,63.083333,1467.0,1179.0,36.0,39.0,0.038927,0.015852,1.862441,0.713367,1.178299,0.133533,0.003576,0.006797,0.105556,0.142379,0.031910,0.052660,0.002265,1.474648,0.222144,0.165649,1.432049,0.172212,0.258180,5,7,0.229885,12,66.0,135.0,68.0,38.0,1,0,0
1214,Sergio Moraes vs Omari Akhmedov,2015-12-10,Omari-Akhmedov,20.116667,241.0,172.0,11.0,12.0,0.014136,0.049710,0.793655,0.185589,0.754279,0.569680,0.006305,0.010269,0.141414,0.080556,0.000000,0.028271,0.000000,0.587786,0.135101,0.070767,0.404086,0.155760,0.233809,3,1,0.564972,4,72.0,185.0,73.0,28.0,1,0,0
964,Elizeu Zaleski dos Santos vs Curtis Millender,2019-03-09,Curtis-Millender,19.283333,365.0,326.0,2.0,9.0,0.000000,0.000000,1.264766,0.196582,1.083625,0.157212,0.003438,0.005465,1.000000,0.119048,0.000000,0.017224,0.000000,0.636929,0.162796,0.465042,1.190683,0.059267,0.014817,3,0,0.534759,3,74.0,170.0,76.0,31.0,1,0,0
171,Nikita Krylov vs Misha Cirkunov,2016-12-10,Nikita-Krylov,20.150000,348.0,257.0,5.0,15.0,0.075251,0.000000,4.024669,0.574431,3.764880,0.990808,0.011869,0.010560,0.100000,0.180952,0.135161,0.043778,0.011863,2.568825,0.839210,0.616634,2.098215,1.371991,0.554463,6,2,0.531915,8,75.0,205.0,77.0,24.0,1,0,0


In [45]:
winners.columns = [c.replace('_w', '') for c in winners]
losers.columns = [c.replace('_l', '') for c in losers]

In [46]:
fighter1 = pd.DataFrame()
fighter1 = fighter1.append(winners[0:777])
fighter1 = fighter1.append(losers[777:])

In [47]:
fighter1

,fight,date,fighter,cumultime,cumulStr_Att_fighter,cumulStr_Att_oponent,cumulTD_Att_fighter,cumulTD_Att_oponent,KDPM,KDPM_ratio,SLPM,SLPM_ratio,SAPM,SAPM_ratio,Str_Acc,Str_Def,TD_Acc,TD_Def,SubPM,PassPM,RevPM,Head_SLPM,Body_SLPM,Leg_SLPM,Dist_SLPM,Clinch_SLPM,Ground_SLPM,wins,losses,implied_prob,prev,height,weight,reach,age,stance_Orthodox,stance_Southpaw,stance_Switch
1512,Leandro Silva vs Charlie Brenneman,2014-11-08,Leandro-Silva,14.000000,127.0,159.0,14.0,5.0,0.000000,0.000000,0.775510,0.159812,0.704082,0.146044,0.009643,0.006409,0.035714,0.200000,0.010204,0.040816,0.000000,0.346939,0.132653,0.295918,0.683673,0.091837,0.000000,0,2,0.609756,2,69.0,155.0,72.0,28.0,1,0,0
294,Jon Jones vs Daniel Cormier,2015-01-03,Jon-Jones,82.200000,1419.0,1168.0,53.0,35.0,0.007096,0.000000,1.539772,0.453875,0.845349,0.194068,0.005770,0.008242,0.141967,0.291667,0.023926,0.051751,0.000000,0.864081,0.314560,0.361131,0.862241,0.246077,0.431453,14,1,0.609756,15,76.0,205.0,84.0,27.0,1,0,0
63,Marlon Vera vs Frankie Saenz,2019-03-23,Marlon-Vera,54.783333,807.0,964.0,25.0,26.0,0.005710,0.000000,1.211761,0.180450,1.440506,0.232817,0.005771,0.005252,0.123333,0.175916,0.021664,0.042691,0.000000,0.652580,0.309733,0.249448,0.780882,0.165348,0.265530,6,4,0.617284,10,68.0,135.0,70.0,26.0,0,0,1
1312,Alex Oliveira vs James Moontasri,2016-07-23,Alex-Oliveira,21.483333,253.0,173.0,20.0,16.0,0.016333,0.000000,1.345555,0.427501,0.989875,0.197760,0.009170,0.015141,0.015625,0.187500,0.013299,0.056231,0.000000,0.888153,0.246254,0.211148,0.772143,0.527448,0.045964,3,2,0.666667,5,71.0,170.0,76.0,28.0,1,0,0
13,Karolina Kowalkiewicz vs Michelle Waterson,2019-03-30,Michelle-Waterson,32.766667,398.0,518.0,14.0,12.0,0.000000,0.000000,1.117159,0.147378,1.591522,0.408006,0.008188,0.004679,0.244048,0.187500,0.006587,0.026159,0.004360,0.531346,0.342690,0.243123,0.625972,0.295647,0.195539,4,2,0.408163,6,63.0,115.0,62.0,33.0,1,0,0
721,Joseph Benavidez vs Dustin Ortiz,2019-01-19,Joseph-Benavidez,92.716667,2030.0,1482.0,53.0,77.0,0.016273,0.000000,1.463462,0.333259,0.873614,0.124406,0.003085,0.006528,0.072216,0.143267,0.011724,0.069795,0.002609,0.835980,0.368797,0.258684,1.123083,0.186038,0.154342,13,3,0.787402,16,64.0,125.0,65.0,34.0,0,1,0
69,Urijah Faber vs Brad Pickett,2016-12-17,Urijah-Faber,91.066667,1641.0,1694.0,69.0,57.0,0.011275,0.000000,1.241575,0.366832,0.958663,0.153747,0.004216,0.005889,0.059213,0.111914,0.022423,0.027406,0.006887,0.890280,0.230164,0.121131,0.792866,0.161124,0.287584,9,6,0.757576,15,66.0,135.0,67.0,37.0,1,0,0
1214,Sergio Moraes vs Omari Akhmedov,2015-12-10,Sergio-Moraes,22.383333,220.0,327.0,17.0,0.0,0.000000,0.000000,0.886933,0.199362,0.890069,0.187206,0.009764,0.007996,0.119430,NaN,0.022536,0.078108,0.000000,0.693928,0.115293,0.077712,0.699962,0.046941,0.140030,3,1,0.444444,4,72.0,170.0,72.0,33.0,1,0,0
964,Elizeu Zaleski dos Santos vs Curtis Millender,2019-03-09,Elizeu-Zaleski-dos-Santos,39.050000,774.0,761.0,26.0,23.0,0.017959,0.000000,1.453954,0.276499,1.147351,0.159301,0.003697,0.005982,0.093590,0.192547,0.005071,0.035750,0.005071,0.945633,0.306452,0.201869,0.847619,0.464244,0.142091,6,1,0.476190,7,71.0,170.0,73.0,32.0,1,0,0
171,Nikita Krylov vs Misha Cirkunov,2016-12-10,Misha-Cirkunov,10.583333,186.0,180.0,13.0,1.0,0.000000,0.000000,2.938102,0.740157,1.405829,0.122405,0.009697,0.011383,0.121795,1.000000,0.066372,0.319057,0.000000,2.289252,0.505866,0.142984,1.791165,0.732157,0.414780,3,0,0.531915,3,75.0,205.0,77.0,29.0,1,0,0


In [48]:
fighter2 = pd.DataFrame()
fighter2 = fighter2.append(losers[0:777])
fighter2 = fighter2.append(winners[777:])

In [49]:
fighter2

,fight,date,fighter,cumultime,cumulStr_Att_fighter,cumulStr_Att_oponent,cumulTD_Att_fighter,cumulTD_Att_oponent,KDPM,KDPM_ratio,SLPM,SLPM_ratio,SAPM,SAPM_ratio,Str_Acc,Str_Def,TD_Acc,TD_Def,SubPM,PassPM,RevPM,Head_SLPM,Body_SLPM,Leg_SLPM,Dist_SLPM,Clinch_SLPM,Ground_SLPM,wins,losses,implied_prob,prev,height,weight,reach,age,stance_Orthodox,stance_Southpaw,stance_Switch
1512,Leandro Silva vs Charlie Brenneman,2014-11-08,Charlie-Brenneman,40.883333,231.0,229.0,48.0,11.0,0.013977,0.000000,0.846688,0.422996,1.338457,0.651842,0.020701,0.019675,0.064608,0.500000,0.013977,0.148752,0.003494,0.405448,0.335118,0.106122,0.234423,0.149987,0.462278,4,6,0.400000,10,70.0,155.0,72.0,33.0,1,0,0
294,Jon Jones vs Daniel Cormier,2015-01-03,Daniel-Cormier,21.200000,325.0,218.0,8.0,5.0,0.035825,0.000000,1.706107,0.723729,0.846836,0.153420,0.006189,0.011381,0.200000,0.600000,0.016035,0.088193,0.000000,1.224436,0.373854,0.107817,0.660546,0.327374,0.718187,4,0,0.400000,4,71.0,235.0,72.0,35.0,1,0,0
63,Marlon Vera vs Frankie Saenz,2019-03-23,Frankie-Saenz,52.900000,879.0,908.0,26.0,76.0,0.004847,0.000000,1.137955,0.137926,1.282121,0.193901,0.004096,0.004561,0.042735,0.066829,0.005401,0.010802,0.002701,0.677344,0.317483,0.143127,0.799906,0.291101,0.046947,5,3,0.400000,8,66.0,135.0,66.0,38.0,1,0,0
1312,Alex Oliveira vs James Moontasri,2016-07-23,James-Moontasri,17.350000,169.0,122.0,0.0,23.0,0.020764,0.000000,1.198715,0.321112,0.816544,0.233373,0.011168,0.016631,NaN,0.124638,0.020463,0.028697,0.000000,0.849117,0.212950,0.136648,0.411018,0.286780,0.500917,2,2,0.347222,4,70.0,170.0,71.0,28.0,0,0,1
13,Karolina Kowalkiewicz vs Michelle Waterson,2019-03-30,Karolina-Kowalkiewicz,49.016667,1400.0,1415.0,22.0,24.0,0.000000,0.000000,1.809788,0.173745,1.916593,0.220659,0.002191,0.003358,0.042929,0.145833,0.000000,0.014572,0.002914,1.259514,0.347847,0.202426,1.322359,0.452455,0.034974,5,3,0.591716,8,63.0,115.0,64.0,33.0,1,0,0
721,Joseph Benavidez vs Dustin Ortiz,2019-01-19,Dustin-Ortiz,75.450000,998.0,881.0,97.0,56.0,0.059087,0.000000,1.359259,0.389158,0.847364,0.164900,0.005982,0.008766,0.036731,0.098335,0.020425,0.058558,0.013959,0.946778,0.257063,0.155418,0.609414,0.220994,0.528851,8,5,0.229885,13,65.0,125.0,65.0,30.0,1,0,0
69,Urijah Faber vs Brad Pickett,2016-12-17,Brad-Pickett,63.083333,1467.0,1179.0,36.0,39.0,0.038927,0.015852,1.862441,0.713367,1.178299,0.133533,0.003576,0.006797,0.105556,0.142379,0.031910,0.052660,0.002265,1.474648,0.222144,0.165649,1.432049,0.172212,0.258180,5,7,0.229885,12,66.0,135.0,68.0,38.0,1,0,0
1214,Sergio Moraes vs Omari Akhmedov,2015-12-10,Omari-Akhmedov,20.116667,241.0,172.0,11.0,12.0,0.014136,0.049710,0.793655,0.185589,0.754279,0.569680,0.006305,0.010269,0.141414,0.080556,0.000000,0.028271,0.000000,0.587786,0.135101,0.070767,0.404086,0.155760,0.233809,3,1,0.564972,4,72.0,185.0,73.0,28.0,1,0,0
964,Elizeu Zaleski dos Santos vs Curtis Millender,2019-03-09,Curtis-Millender,19.283333,365.0,326.0,2.0,9.0,0.000000,0.000000,1.264766,0.196582,1.083625,0.157212,0.003438,0.005465,1.000000,0.119048,0.000000,0.017224,0.000000,0.636929,0.162796,0.465042,1.190683,0.059267,0.014817,3,0,0.534759,3,74.0,170.0,76.0,31.0,1,0,0
171,Nikita Krylov vs Misha Cirkunov,2016-12-10,Nikita-Krylov,20.150000,348.0,257.0,5.0,15.0,0.075251,0.000000,4.024669,0.574431,3.764880,0.990808,0.011869,0.010560,0.100000,0.180952,0.135161,0.043778,0.011863,2.568825,0.839210,0.616634,2.098215,1.371991,0.554463,6,2,0.531915,8,75.0,205.0,77.0,24.0,1,0,0


In [50]:
fighter1.to_csv('fighter1.csv')
fighter2.to_csv('fighter2.csv')

In [51]:
fighter1 = fighter1.add_suffix('_1')
fighter2 = fighter2.add_suffix('_2')

In [52]:
fighter1.rename(columns={'date_1': 'date'}, inplace=True)
fighter2.rename(columns={'date_2': 'date'}, inplace=True)

fighter1.rename(columns={'fight_1': 'fight'}, inplace=True)
fighter2.rename(columns={'fight_2': 'fight'}, inplace=True)

In [53]:
fighter1

,fight,date,fighter_1,cumultime_1,cumulStr_Att_fighter_1,cumulStr_Att_oponent_1,cumulTD_Att_fighter_1,cumulTD_Att_oponent_1,KDPM_1,KDPM_ratio_1,SLPM_1,SLPM_ratio_1,SAPM_1,SAPM_ratio_1,Str_Acc_1,Str_Def_1,TD_Acc_1,TD_Def_1,SubPM_1,PassPM_1,RevPM_1,Head_SLPM_1,Body_SLPM_1,Leg_SLPM_1,Dist_SLPM_1,Clinch_SLPM_1,Ground_SLPM_1,wins_1,losses_1,implied_prob_1,prev_1,height_1,weight_1,reach_1,age_1,stance_Orthodox_1,stance_Southpaw_1,stance_Switch_1
1512,Leandro Silva vs Charlie Brenneman,2014-11-08,Leandro-Silva,14.000000,127.0,159.0,14.0,5.0,0.000000,0.000000,0.775510,0.159812,0.704082,0.146044,0.009643,0.006409,0.035714,0.200000,0.010204,0.040816,0.000000,0.346939,0.132653,0.295918,0.683673,0.091837,0.000000,0,2,0.609756,2,69.0,155.0,72.0,28.0,1,0,0
294,Jon Jones vs Daniel Cormier,2015-01-03,Jon-Jones,82.200000,1419.0,1168.0,53.0,35.0,0.007096,0.000000,1.539772,0.453875,0.845349,0.194068,0.005770,0.008242,0.141967,0.291667,0.023926,0.051751,0.000000,0.864081,0.314560,0.361131,0.862241,0.246077,0.431453,14,1,0.609756,15,76.0,205.0,84.0,27.0,1,0,0
63,Marlon Vera vs Frankie Saenz,2019-03-23,Marlon-Vera,54.783333,807.0,964.0,25.0,26.0,0.005710,0.000000,1.211761,0.180450,1.440506,0.232817,0.005771,0.005252,0.123333,0.175916,0.021664,0.042691,0.000000,0.652580,0.309733,0.249448,0.780882,0.165348,0.265530,6,4,0.617284,10,68.0,135.0,70.0,26.0,0,0,1
1312,Alex Oliveira vs James Moontasri,2016-07-23,Alex-Oliveira,21.483333,253.0,173.0,20.0,16.0,0.016333,0.000000,1.345555,0.427501,0.989875,0.197760,0.009170,0.015141,0.015625,0.187500,0.013299,0.056231,0.000000,0.888153,0.246254,0.211148,0.772143,0.527448,0.045964,3,2,0.666667,5,71.0,170.0,76.0,28.0,1,0,0
13,Karolina Kowalkiewicz vs Michelle Waterson,2019-03-30,Michelle-Waterson,32.766667,398.0,518.0,14.0,12.0,0.000000,0.000000,1.117159,0.147378,1.591522,0.408006,0.008188,0.004679,0.244048,0.187500,0.006587,0.026159,0.004360,0.531346,0.342690,0.243123,0.625972,0.295647,0.195539,4,2,0.408163,6,63.0,115.0,62.0,33.0,1,0,0
721,Joseph Benavidez vs Dustin Ortiz,2019-01-19,Joseph-Benavidez,92.716667,2030.0,1482.0,53.0,77.0,0.016273,0.000000,1.463462,0.333259,0.873614,0.124406,0.003085,0.006528,0.072216,0.143267,0.011724,0.069795,0.002609,0.835980,0.368797,0.258684,1.123083,0.186038,0.154342,13,3,0.787402,16,64.0,125.0,65.0,34.0,0,1,0
69,Urijah Faber vs Brad Pickett,2016-12-17,Urijah-Faber,91.066667,1641.0,1694.0,69.0,57.0,0.011275,0.000000,1.241575,0.366832,0.958663,0.153747,0.004216,0.005889,0.059213,0.111914,0.022423,0.027406,0.006887,0.890280,0.230164,0.121131,0.792866,0.161124,0.287584,9,6,0.757576,15,66.0,135.0,67.0,37.0,1,0,0
1214,Sergio Moraes vs Omari Akhmedov,2015-12-10,Sergio-Moraes,22.383333,220.0,327.0,17.0,0.0,0.000000,0.000000,0.886933,0.199362,0.890069,0.187206,0.009764,0.007996,0.119430,NaN,0.022536,0.078108,0.000000,0.693928,0.115293,0.077712,0.699962,0.046941,0.140030,3,1,0.444444,4,72.0,170.0,72.0,33.0,1,0,0
964,Elizeu Zaleski dos Santos vs Curtis Millender,2019-03-09,Elizeu-Zaleski-dos-Santos,39.050000,774.0,761.0,26.0,23.0,0.017959,0.000000,1.453954,0.276499,1.147351,0.159301,0.003697,0.005982,0.093590,0.192547,0.005071,0.035750,0.005071,0.945633,0.306452,0.201869,0.847619,0.464244,0.142091,6,1,0.476190,7,71.0,170.0,73.0,32.0,1,0,0
171,Nikita Krylov vs Misha Cirkunov,2016-12-10,Misha-Cirkunov,10.583333,186.0,180.0,13.0,1.0,0.000000,0.000000,2.938102,0.740157,1.405829,0.122405,0.009697,0.011383,0.121795,1.000000,0.066372,0.319057,0.000000,2.289252,0.505866,0.142984,1.791165,0.732157,0.414780,3,0,0.531915,3,75.0,205.0,77.0,29.0,1,0,0


In [54]:
fighter2

,fight,date,fighter_2,cumultime_2,cumulStr_Att_fighter_2,cumulStr_Att_oponent_2,cumulTD_Att_fighter_2,cumulTD_Att_oponent_2,KDPM_2,KDPM_ratio_2,SLPM_2,SLPM_ratio_2,SAPM_2,SAPM_ratio_2,Str_Acc_2,Str_Def_2,TD_Acc_2,TD_Def_2,SubPM_2,PassPM_2,RevPM_2,Head_SLPM_2,Body_SLPM_2,Leg_SLPM_2,Dist_SLPM_2,Clinch_SLPM_2,Ground_SLPM_2,wins_2,losses_2,implied_prob_2,prev_2,height_2,weight_2,reach_2,age_2,stance_Orthodox_2,stance_Southpaw_2,stance_Switch_2
1512,Leandro Silva vs Charlie Brenneman,2014-11-08,Charlie-Brenneman,40.883333,231.0,229.0,48.0,11.0,0.013977,0.000000,0.846688,0.422996,1.338457,0.651842,0.020701,0.019675,0.064608,0.500000,0.013977,0.148752,0.003494,0.405448,0.335118,0.106122,0.234423,0.149987,0.462278,4,6,0.400000,10,70.0,155.0,72.0,33.0,1,0,0
294,Jon Jones vs Daniel Cormier,2015-01-03,Daniel-Cormier,21.200000,325.0,218.0,8.0,5.0,0.035825,0.000000,1.706107,0.723729,0.846836,0.153420,0.006189,0.011381,0.200000,0.600000,0.016035,0.088193,0.000000,1.224436,0.373854,0.107817,0.660546,0.327374,0.718187,4,0,0.400000,4,71.0,235.0,72.0,35.0,1,0,0
63,Marlon Vera vs Frankie Saenz,2019-03-23,Frankie-Saenz,52.900000,879.0,908.0,26.0,76.0,0.004847,0.000000,1.137955,0.137926,1.282121,0.193901,0.004096,0.004561,0.042735,0.066829,0.005401,0.010802,0.002701,0.677344,0.317483,0.143127,0.799906,0.291101,0.046947,5,3,0.400000,8,66.0,135.0,66.0,38.0,1,0,0
1312,Alex Oliveira vs James Moontasri,2016-07-23,James-Moontasri,17.350000,169.0,122.0,0.0,23.0,0.020764,0.000000,1.198715,0.321112,0.816544,0.233373,0.011168,0.016631,NaN,0.124638,0.020463,0.028697,0.000000,0.849117,0.212950,0.136648,0.411018,0.286780,0.500917,2,2,0.347222,4,70.0,170.0,71.0,28.0,0,0,1
13,Karolina Kowalkiewicz vs Michelle Waterson,2019-03-30,Karolina-Kowalkiewicz,49.016667,1400.0,1415.0,22.0,24.0,0.000000,0.000000,1.809788,0.173745,1.916593,0.220659,0.002191,0.003358,0.042929,0.145833,0.000000,0.014572,0.002914,1.259514,0.347847,0.202426,1.322359,0.452455,0.034974,5,3,0.591716,8,63.0,115.0,64.0,33.0,1,0,0
721,Joseph Benavidez vs Dustin Ortiz,2019-01-19,Dustin-Ortiz,75.450000,998.0,881.0,97.0,56.0,0.059087,0.000000,1.359259,0.389158,0.847364,0.164900,0.005982,0.008766,0.036731,0.098335,0.020425,0.058558,0.013959,0.946778,0.257063,0.155418,0.609414,0.220994,0.528851,8,5,0.229885,13,65.0,125.0,65.0,30.0,1,0,0
69,Urijah Faber vs Brad Pickett,2016-12-17,Brad-Pickett,63.083333,1467.0,1179.0,36.0,39.0,0.038927,0.015852,1.862441,0.713367,1.178299,0.133533,0.003576,0.006797,0.105556,0.142379,0.031910,0.052660,0.002265,1.474648,0.222144,0.165649,1.432049,0.172212,0.258180,5,7,0.229885,12,66.0,135.0,68.0,38.0,1,0,0
1214,Sergio Moraes vs Omari Akhmedov,2015-12-10,Omari-Akhmedov,20.116667,241.0,172.0,11.0,12.0,0.014136,0.049710,0.793655,0.185589,0.754279,0.569680,0.006305,0.010269,0.141414,0.080556,0.000000,0.028271,0.000000,0.587786,0.135101,0.070767,0.404086,0.155760,0.233809,3,1,0.564972,4,72.0,185.0,73.0,28.0,1,0,0
964,Elizeu Zaleski dos Santos vs Curtis Millender,2019-03-09,Curtis-Millender,19.283333,365.0,326.0,2.0,9.0,0.000000,0.000000,1.264766,0.196582,1.083625,0.157212,0.003438,0.005465,1.000000,0.119048,0.000000,0.017224,0.000000,0.636929,0.162796,0.465042,1.190683,0.059267,0.014817,3,0,0.534759,3,74.0,170.0,76.0,31.0,1,0,0
171,Nikita Krylov vs Misha Cirkunov,2016-12-10,Nikita-Krylov,20.150000,348.0,257.0,5.0,15.0,0.075251,0.000000,4.024669,0.574431,3.764880,0.990808,0.011869,0.010560,0.100000,0.180952,0.135161,0.043778,0.011863,2.568825,0.839210,0.616634,2.098215,1.371991,0.554463,6,2,0.531915,8,75.0,205.0,77.0,24.0,1,0,0


In [55]:

w1 = [1] * 777
w2 = [2] * 778
w = w1 + w2

In [56]:

len(w)

1555

In [57]:
fighter1['winner'] = w
fighter2['winner'] = w

In [62]:
fighter1.to_csv('fighter1.csv')
fighter2.to_csv('fighter2.csv')

In [58]:
finaldf_formodels = pd.merge(fighter1, fighter2, how='inner', on=['fight', 'date', 'winner']).drop_duplicates()

In [59]:
finaldf_formodels

,fight,date,fighter_1,cumultime_1,cumulStr_Att_fighter_1,cumulStr_Att_oponent_1,cumulTD_Att_fighter_1,cumulTD_Att_oponent_1,KDPM_1,KDPM_ratio_1,SLPM_1,SLPM_ratio_1,SAPM_1,SAPM_ratio_1,Str_Acc_1,Str_Def_1,TD_Acc_1,TD_Def_1,SubPM_1,PassPM_1,RevPM_1,Head_SLPM_1,Body_SLPM_1,Leg_SLPM_1,Dist_SLPM_1,Clinch_SLPM_1,Ground_SLPM_1,wins_1,losses_1,implied_prob_1,prev_1,height_1,weight_1,reach_1,age_1,stance_Orthodox_1,stance_Southpaw_1,stance_Switch_1,winner,fighter_2,cumultime_2,cumulStr_Att_fighter_2,cumulStr_Att_oponent_2,cumulTD_Att_fighter_2,cumulTD_Att_oponent_2,KDPM_2,KDPM_ratio_2,SLPM_2,SLPM_ratio_2,SAPM_2,SAPM_ratio_2,Str_Acc_2,Str_Def_2,TD_Acc_2,TD_Def_2,SubPM_2,PassPM_2,RevPM_2,Head_SLPM_2,Body_SLPM_2,Leg_SLPM_2,Dist_SLPM_2,Clinch_SLPM_2,Ground_SLPM_2,wins_2,losses_2,implied_prob_2,prev_2,height_2,weight_2,reach_2,age_2,stance_Orthodox_2,stance_Southpaw_2,stance_Switch_2
0,Leandro Silva vs Charlie Brenneman,2014-11-08,Leandro-Silva,14.000000,127.0,159.0,14.0,5.0,0.000000,0.000000,0.775510,0.159812,0.704082,0.146044,0.009643,0.006409,0.035714,0.200000,0.010204,0.040816,0.000000,0.346939,0.132653,0.295918,0.683673,0.091837,0.000000,0,2,0.609756,2,69.0,155.0,72.0,28.0,1,0,0,1,Charlie-Brenneman,40.883333,231.0,229.0,48.0,11.0,0.013977,0.000000,0.846688,0.422996,1.338457,0.651842,0.020701,0.019675,0.064608,0.500000,0.013977,0.148752,0.003494,0.405448,0.335118,0.106122,0.234423,0.149987,0.462278,4,6,0.400000,10,70.0,155.0,72.0,33.0,1,0,0
1,Jon Jones vs Daniel Cormier,2015-01-03,Jon-Jones,82.200000,1419.0,1168.0,53.0,35.0,0.007096,0.000000,1.539772,0.453875,0.845349,0.194068,0.005770,0.008242,0.141967,0.291667,0.023926,0.051751,0.000000,0.864081,0.314560,0.361131,0.862241,0.246077,0.431453,14,1,0.609756,15,76.0,205.0,84.0,27.0,1,0,0,1,Daniel-Cormier,21.200000,325.0,218.0,8.0,5.0,0.035825,0.000000,1.706107,0.723729,0.846836,0.153420,0.006189,0.011381,0.200000,0.600000,0.016035,0.088193,0.000000,1.224436,0.373854,0.107817,0.660546,0.327374,0.718187,4,0,0.400000,4,71.0,235.0,72.0,35.0,1,0,0
2,Marlon Vera vs Frankie Saenz,2019-03-23,Marlon-Vera,54.783333,807.0,964.0,25.0,26.0,0.005710,0.000000,1.211761,0.180450,1.440506,0.232817,0.005771,0.005252,0.123333,0.175916,0.021664,0.042691,0.000000,0.652580,0.309733,0.249448,0.780882,0.165348,0.265530,6,4,0.617284,10,68.0,135.0,70.0,26.0,0,0,1,1,Frankie-Saenz,52.900000,879.0,908.0,26.0,76.0,0.004847,0.000000,1.137955,0.137926,1.282121,0.193901,0.004096,0.004561,0.042735,0.066829,0.005401,0.010802,0.002701,0.677344,0.317483,0.143127,0.799906,0.291101,0.046947,5,3,0.400000,8,66.0,135.0,66.0,38.0,1,0,0
3,Alex Oliveira vs James Moontasri,2016-07-23,Alex-Oliveira,21.483333,253.0,173.0,20.0,16.0,0.016333,0.000000,1.345555,0.427501,0.989875,0.197760,0.009170,0.015141,0.015625,0.187500,0.013299,0.056231,0.000000,0.888153,0.246254,0.211148,0.772143,0.527448,0.045964,3,2,0.666667,5,71.0,170.0,76.0,28.0,1,0,0,1,James-Moontasri,17.350000,169.0,122.0,0.0,23.0,0.020764,0.000000,1.198715,0.321112,0.816544,0.233373,0.011168,0.016631,NaN,0.124638,0.020463,0.028697,0.000000,0.849117,0.212950,0.136648,0.411018,0.286780,0.500917,2,2,0.347222,4,70.0,170.0,71.0,28.0,0,0,1
4,Karolina Kowalkiewicz vs Michelle Waterson,2019-03-30,Michelle-Waterson,32.766667,398.0,518.0,14.0,12.0,0.000000,0.000000,1.117159,0.147378,1.591522,0.408006,0.008188,0.004679,0.244048,0.187500,0.006587,0.026159,0.004360,0.531346,0.342690,0.243123,0.625972,0.295647,0.195539,4,2,0.408163,6,63.0,115.0,62.0,33.0,1,0,0,1,Karolina-Kowalkiewicz,49.016667,1400.0,1415.0,22.0,24.0,0.000000,0.000000,1.809788,0.173745,1.916593,0.220659,0.002191,0.003358,0.042929,0.145833,0.000000,0.014572,0.002914,1.259514,0.347847,0.202426,1.322359,0.452455,0.034974,5,3,0.591716,8,63.0,115.0,64.0,33.0,1,0,0
5,Joseph Benavidez vs Dustin Ortiz,2019-01-19,Joseph-Benavidez,92.716667,2030.0,1482.0,53.0,77.0,0.016273,0.000000,1.463462,0.333259,0.873614,0.124406,0.003085,0.006528,0.072216,0.143267,0.011724,0.069795,0.002609,0.835980,0.368797,0.258684,1.123083,0.186038,0.154342,13,3

In [60]:
bloop = finaldf_formodels.drop(['fight', 'date', 'fighter_1', 'fighter_2'], axis=1)

In [61]:
bloop.columns

Index(['cumultime_1', 'cumulStr_Att_fighter_1', 'cumulStr_Att_oponent_1',
       'cumulTD_Att_fighter_1', 'cumulTD_Att_oponent_1', 'KDPM_1',
       'KDPM_ratio_1', 'SLPM_1', 'SLPM_ratio_1', 'SAPM_1', 'SAPM_ratio_1',
       'Str_Acc_1', 'Str_Def_1', 'TD_Acc_1', 'TD_Def_1', 'SubPM_1', 'PassPM_1',
       'RevPM_1', 'Head_SLPM_1', 'Body_SLPM_1', 'Leg_SLPM_1', 'Dist_SLPM_1',
       'Clinch_SLPM_1', 'Ground_SLPM_1', 'wins_1', 'losses_1',
       'implied_prob_1', 'prev_1', 'height_1', 'weight_1', 'reach_1', 'age_1',
       'stance_Orthodox_1', 'stance_Southpaw_1', 'stance_Switch_1', 'winner',
       'cumultime_2', 'cumulStr_Att_fighter_2', 'cumulStr_Att_oponent_2',
       'cumulTD_Att_fighter_2', 'cumulTD_Att_oponent_2', 'KDPM_2',
       'KDPM_ratio_2', 'SLPM_2', 'SLPM_ratio_2', 'SAPM_2', 'SAPM_ratio_2',
       'Str_Acc_2', 'Str_Def_2', 'TD_Acc_2', 'TD_Def_2', 'SubPM_2', 'PassPM_2',
       'RevPM_2', 'Head_SLPM_2', 'Body_SLPM_2', 'Leg_SLPM_2', 'Dist_SLPM_2',
       'Clinch_SLPM_2', 'Ground_SL

In [36]:
bloop.fillna(0, inplace=True)

B = bloop
for col in B.columns:
    B[col] = (B[col]-min(B[col]))/ (max(B[col]) - min(B[col])) #subtract the minimum and divide by the range forcing a scale of 0 to 1 for each feature
B

M = B.sample(frac=1)
M

M['win'] = M['winner']
M.drop(columns=['winner'], inplace=True)
M

,cumultime_1,cumulStr_Att_fighter_1,cumulStr_Att_oponent_1,cumulTD_Att_fighter_1,cumulTD_Att_oponent_1,KDPM_1,KDPM_ratio_1,SLPM_1,SLPM_ratio_1,SAPM_1,SAPM_ratio_1,Str_Acc_1,Str_Def_1,TD_Acc_1,TD_Def_1,SubPM_1,PassPM_1,RevPM_1,Head_SLPM_1,Body_SLPM_1,Leg_SLPM_1,Dist_SLPM_1,Clinch_SLPM_1,Ground_SLPM_1,wins_1,losses_1,implied_prob_1,prev_1,height_1,weight_1,reach_1,stance_Orthodox_1,stance_Southpaw_1,stance_Switch_1,cumultime_2,cumulStr_Att_fighter_2,cumulStr_Att_oponent_2,cumulTD_Att_fighter_2,cumulTD_Att_oponent_2,KDPM_2,KDPM_ratio_2,SLPM_2,SLPM_ratio_2,SAPM_2,SAPM_ratio_2,Str_Acc_2,Str_Def_2,TD_Acc_2,TD_Def_2,SubPM_2,PassPM_2,RevPM_2,Head_SLPM_2,Body_SLPM_2,Leg_SLPM_2,Dist_SLPM_2,Clinch_SLPM_2,Ground_SLPM_2,wins_2,losses_2,implied_prob_2,prev_2,height_2,weight_2,reach_2,stance_Orthodox_2,stance_Southpaw_2,stance_Switch_2,win
2468,0.235162,0.184668,0.196988,0.057803,0.056497,0.018290,0.244565,0.119136,0.092903,0.159486,0.047719,0.039127,0.023904,0.290000,0.175000,0.000000,0.050072,0.000000,0.101307,0.162009,0.202287,0.164247,0.049997,0.023946,0.260870,0.153846,0.216157,0.200000,0.500000,0.366667,0.869048,0.0,1.0,0.0,0.251756,0.122600,0.142857,0.044199,0.090395,0.007956,0.000000,0.076554,0.052192,0.090913,0.270993,0.146168,0.053934,0.562500,0.156250,0.044683,0.065662,0.068636,0.051251,0.080449,0.041039,0.080157,0.025055,0.019875,0.260870,0.142857,0.766099,0.206897,0.478261,0.366667,0.500000,0.0,0.0,1.0,1.0
321,0.217881,0.129874,0.194837,0.098266,0.163842,0.025423,0.262214,0.075471,0.070493,0.205026,0.081502,0.043686,0.030987,0.067227,0.132184,0.099602,0.063033,0.015772,0.098445,0.029124,0.065992,0.121401,0.015784,0.020129,0.217391,0.307692,0.341321,0.233333,0.409091,1.000000,0.857143,1.0,0.0,0.0,0.082101,0.040184,0.061131,0.011050,0.050847,0.068647,0.000000,0.244934,0.256678,0.347409,0.923443,0.358996,0.126247,0.500000,0.407407,0.000000,0.027160,0.000000,0.154710,0.296157,0.083884,0.128351,0.199566,0.095908,0.130435,0.285714,0.256712,0.172414,0.695652,0.993333,0.833333,1.0,0.0,0.0,0.0
921,0.436943,0.351949,0.378611,0.294798,0.135593,0.047111,0.000000,0.154203,0.143891,0.173343,0.043509,0.040926,0.024311,0.099362,0.130704,0.000000,0.050755,0.000000,0.169734,0.079956,0.154670,0.168995,0.029285,0.127630,0.521739,0.230769,0.667013,0.433333,0.681818,0.833333,0.952381,1.0,0.0,0.0,0.391003,0.268236,0.291599,0.198895,0.158192,0.011325,0.000000,0.135533,0.113329,0.129773,0.044956,0.090064,0.046577,0.183102,0.198172,0.048570,0.084519,0.000000,0.118646,0.071555,0.045955,0.092951,0.083721,0.051555,0.478261,0.071429,0.327821,0.344828,0.478261,0.800000,0.500000,1.0,0.0,0.0,0.0
3050,0.351186,0.298736,0.347572,0.236994,0.158192,0.001702,0.000000,0.113194,0.099292,0.103731,0.022369,0.025858,0.018215,0.077236,0.148810,0.015637,0.057532,0.000000,0.096004,0.135943,0.230499,0.157241,0.050344,0.020178,0.260870,0.230769,0.395369,0.233333,0.000000,0.000000,0.714286,1.0,0.0,0.0,0.041418,0.041976,0.064073,0.022099,0.016949,0.000000,0.000000,0.148057,0.136563,0.149726,0.069063,0.085107,0.020885,0.083333,0.500000,0.172263,0.057213,0.000000,0.127112,0.093532,0.039615,0.065068,0.027841,0.121683,0.086957,0.000000,0.552361,0.000000,0.173913,0.000000,0.125000,0.0,0.0,1.0,1.0
2757,0.102501,0.115648,0.124770,0.075145,0.033898,0.027538,0.000000,0.147547,0.253690,0.196095,0.037614,0.031581,0.019077,0.012821,0.277778,0.009831,0.017641,0.000000,0.142968,0.123590,0.214412,0.196107,0.027448,0.066613,0.130435,0.076923,0.470205,0.066667,0.636364,0.366667,0.916667,1.0,0.0,0.0,0.287407,0.200921,0.184374,0.292818,0.220339,0.004377,0.000000,0.118631,0.143172,0.106708,0.068748,0.059022,0.061590,0.060131,0.103114,0.016010,0.048965,0.000000,0.088611,0.095843,0.056488,0.109208,0.074250,0.020260,0.304348,0.142857,0.535031,0.241379,0.652174,0.366667,0.833333,1.0,0.0,0.0,1.0
1234,0.342170,0.185985,0.163491,0.248555,0.090395,0.130223,0.000000,0.201437,0.279758,0.123006,0.052097,0.097581,0.095378,0.110659,0.531250,0.050266,0.143720,0.009493,0.205542,0.120274,0.240428,0.200248,0.09279